# Trame d'exploration d'un éventuel article

## Positionnement du papier

- de nombreux épisodes d'insistance sur l'innovation thérapeutique : myopathies (Téléthon), cancers (médecine de précision), etc. qui participent à construire certains modèles de biocommunicabilités autour des médicaments*, renforçant une économie des promesses autour des nouveaux médicaments
- l'épidémie de COVID a conduit à mettre en tension à la fois l'innovation (médicaments, vaccins) et le repositionnement thérapeutique, avec une évolution avec l'échec de l'HC et de l'ivermectine, et réussite des vaccins ARN
- pour les vaccins une des grandes clés de réticence est la nouveauté (perception du risque) - H1N1
- un discours plus ou moins critique sur le rôle des industriels dans le développement des médicaments, qui ont intérêt à développer de nouveaux médicaments (littérature sur la construction des marchés pharmaceutiques + brevets)

Deux questions :

1. Quels sont les facteurs amenant à être sensible aux promesses de la nouveauté
2. Quels sont les facteurs amenant à faire confiance aux industriels pour l'innovation 

Questions supplémentaires : évolution pendant la pandémie ? (on a qu'une partie des questions posées sur les deux vagues);

Objectif du papier : petite publication pour commencer à problématiser le rapport à la nouveauté dans l'innovation biomédicale

Journal visé : PLOS One ?

Participant au papier : 
- Daniela
- Julien
- Émilien
- Jeremy
- Laetitia ?
- Léo ?

## Méthodologie

Deux vagues d'enquête de HLS19 :
- 25052020
- 04012021

Deux questions d'intérêt :
- RM6 Les nouveaux médicaments sont toujours plus efficaces que les anciens
- RM8 Il faut financer en priorité la recherche sur les nouvelles maladies par rapport à celles plus anciennes

Des questions complémentaires :
- RM3 Il faut laisser aux industriels la responsabilité de développer les nouveaux médicaments
- RM4 L'Etat doit intervenir directement pour développer un nouveau médicament quand la maladie est grave

## Quelques questions

- quelle place donner à l'évolution sur un an, comparaison des variables, lesquelles pourraient être intéressantes ?
- il y a les revenues (data2["D16"]) que pour la deuxième vague + certaines questions ont évolué sur la deuxième vague
- se concentrer uniquement sur la seconde vague ?
- une question d'importance pour la recherche clinique



Ajouter les éléments sur la recherche etc.

In [1]:
# Chargement des bibliothèques
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from patsy import dmatrices
import pyshs
import numpy as np, scipy.stats as st
from scipy import stats
import statsmodels.formula.api as smf
import pyshs
%matplotlib inline

# Chargement des données 
data = pd.read_excel("HLS19 Vague2/bdd2.xlsx")

#Variables pour l'article
data["efficacitenouveaumedF"] = data["RM_6._scale"].replace({1:"1 - Totally agree",
            2:"2 - Rather agree",3:"3 - Rather disagree",4:"4 - Totally disagree"}).fillna("NA")
data["efficacitenouveaumed"] = data["RM_6._scale"].apply(lambda x : "Accord" if x<=2 else "Pas d'accord")
data["efficacitenouveaumedB"] = data["efficacitenouveaumed"].replace({"Accord":1,"Pas d'accord":0})

data["prioritenouveau"] = data["RM_8._scale"].apply(lambda x : "Accord" if x<=2 else "Pas d'accord")
data["prioritenouveauF"] = data["RM_6._scale"].replace({1:"1 - Totally agree",
            2:"2 - Rather agree",3:"3 - Rather disagree",4:"4 - Totally disagree"}).fillna("NA")


def reco(x):
    if x<4:
        return "1 - Below BD"
    if x == 4:
        return "2 - BD"
    else:
        return "3 - Above BD"

data["dip"] = data["DET6"].apply(reco)


data["zones"] = data["FRINSEEMKTSIZE"].replace({1:"Rural",2:"Urban <20k",3:"Urban <100k",4:"Urban >100k",5:"Paris"})
data["sexe"] = data["resp_gender"]
data["age"] = pd.cut(data["resp_age"],[0,25,45,65,100],labels = ["[0-25[","[25-45[","[45-65[","[65+"])
data["age2"] = pd.cut(data["resp_age"],[0,35,45,55,65,100],
                labels = ["[18-35[","[35-45[","[45-55[","[55-65[","[65-75]"])

data["rechercheinfo"] = data["HI1"].replace({1:"Yes",2:"No"})
data["DET8"] = data["DET8"].replace({1:"Yes",2:"No"})

data["EtatSante"] = data["HSTAT1"].replace({1:"1 - Good",2:"1 - Good",3:"2 - Average",4:"3 - Bad",5:"3 - Bad"})

data["confIndustriels"] = data['CONF_6._scale'].replace({1:"1 - Yes",2:"1 - Yes",3:"2 - No",4:"2 - No"})
data["confMedecins"] = data['CONF_4._scale'].replace({1:"1 - Yes",2:"1 - Yes",3:"2 - No",4:"2 - No"})
data["confScientifiques"] = data['CONF_3._scale'].replace({1:"1 - Yes",2:"1 - Yes",3:"2 - No",4:"2 - No"})
data["confPolitiques"] = data['CONF_2._scale'].replace({1:"1 - Yes",2:"1 - Yes",3:"2 - No",4:"2 - No"})


# Indicateur de position sociale
data["def"] = data.apply(lambda x : "Yes" if (x["DET10"]>2) or (x["DET11"]>2) else "No",axis=1)
data["DEF"] = data["def"]

# Indicateur de lité
def reco(x):
    if pd.isnull(x):
        return None
    if x in [1,2]:
        return 1
    return 0

c = ["HL1_"+str(i)+"._scale" for i in range(1,9) if not i in [4,6]] + \
    ["HL2_"+str(i)+"._scale" for i in range(1,8) if i!=5]+ \
     ["HL3_"+str(i)+"._scale" for i in range(1,8) if not i in [1,5,7]]

for i in c :
    data[i+"_reco"] = data[i].apply(reco)
    
data["PERCEPTCORONA"] = data["COV3"].replace({1:"Yes",2:"Some",3:"Some",4:"No"})
data["lit"] = data[[i+"_reco" for i in c]].sum(axis=1)
data["lit_num"] = data["lit"]

def reco(x):
    if pd.isnull(x):
        return None
    if x<9:
        return "3 - Inadequate"
    if x<13:
        return "2 - Problematic"
    else:
        return "1 - Adequate"
data["lit_scale"] = data["lit"].apply(reco)

data["consult"] = data["HCUT3"]>0
data["formationsante"] = data["DET8"]

data["placeIndustriels"] = data['RM_3._scale'].replace({1:"Agree",2:"Agree",3:"Disagree",4:"Disagree"}).fillna("NA")
data["queMedecins"] = data['RM_4._scale'].replace({1:"Agree",2:"Agree",3:"Disagree",4:"Disagree"}).fillna("NA")
data["chomagePlusImportant"] = data['RM_5._scale'].replace({1:"Agree",2:"Agree",3:"Disagree",4:"Disagree"}).fillna("NA")
data["avisCitoyens"] = data['RM_7._scale'].replace({1:"Agree",2:"Agree",3:"Disagree",4:"Disagree"}).fillna("NA")
data["prioriteNouvellesMaladies"] = data['RM_8._scale'].replace({1:"Agree",2:"Agree",3:"Disagree",4:"Disagree"}).fillna("NA")

data["rev"] = data["D16"].replace({1:"1 - Moins de 1000",2:"1 - Moins de 1000",3:"1 - Moins de 1000",
                     4:"2 - Moins de 2000",5:"2 - Moins de 2000",6:"2 - Moins de 2000",
                     7:"3 - Moins de 5000",8:"3 - Moins de 5000",9:"3 - Moins de 5000",
                     10:"4 - Plus de 5000",11:"4 - Plus de 5000",12:"NSPR"
                    
                    })

data["ConnaissanceEC"] = data["EC0"].replace({3:"3-Somewhat familiar with what CTs are", 5:"5-Extremely familiar with what CTs are", 
        2:"2-Only know the term, CTs", 4:"4-Very familiar with what CTs are", 1:"1-Never heard of CTs before today"})
data["Phase"] = data["EC3"].replace({1:"Yes",2:"No"}).fillna("NA")


data["respindustriels"] = data["RM_3._scale"].replace({1:"1 - Totally agree",
            2:"2 - Rather agree",3:"3 - Rather disagree",4:"4 - Totally disagree"}).fillna("NA")
data["respindustrielsD"] = data["RM_3._scale"].replace({1:"1 - Agree",
            2:"1 - Agree",3:"2 - Disagree",4:"2 - Disagree"}).fillna("NA")

data["respetat"] = data["RM_4._scale"].replace({1:"1 - Totally agree",
            2:"2 - Rather agree",3:"3 - Rather disagree",4:"4 - Totally disagree"}).fillna("NA")
data["respetatD"] = data["RM_4._scale"].replace({1:"1 - Agree",
            2:"1 - Agree",3:"2 - Disagree",4:"2 - Disagree"}).fillna("NA")


### Définition des variables d'intérêts et de leurs noms

In [67]:
ind_var = {
           "age2":"Age",
           "sexe":"Sexe",
           "dip":"Diplome",
           "zones":"Zones",
           "lit_scale":"Littéracie",
           "DEF":"Défavorisé",
           "confScientifiques":"Confiance scientifiques",
           "confMedecins":"Confiance médecins",
           "confPolitiques":"Confiance politiques",
           "confIndustriels":"Confiance industriels",
           "rechercheinfo":"Recherche info",
           "EtatSante":"État Santé",
           "consult":"Consultation médecin",
           "formationsante":"Formation santé",
          # "Attitude":"Attitude essais cliniques",
           "PERCEPTCORONA":"Perception Corona",
           "rev":"Revenus"
          }

dep_var = {
    "efficacitenouveaumedoc":"Nouveaux médicaments plus efficaces",
    "prioriteNouvellesMaladies":"Recherche priorité nouvelles maladies",
    "respindustriels":"Industriels responsables développement",
    "respetat":"Etat responsable développement"
}

### Comparaison des vagues

Se poser la question si ça a évolué durant l'épidémie. TL;Dr > Non

In [70]:
t_all = {}
for i in ind_var:
    t_all[ind_var[i]] = pyshs.tableau_croise(data,"Vague",i,weight="weight").T.drop("All")
t_all = pd.concat(t_all)
t_all

1              2  \
Age                     [18-35[            306.9 (30.6%)  306.0 (30.6%)   
                        [35-45[            189.6 (18.9%)  189.0 (18.9%)   
                        [45-55[            195.6 (19.5%)  195.0 (19.5%)   
                        [55-65[            182.5 (18.2%)  182.0 (18.2%)   
                        [65-75]            128.4 (12.8%)  128.0 (12.8%)   
Sexe                    female             515.5 (51.4%)  514.0 (51.4%)   
                        male               487.5 (48.6%)  486.0 (48.6%)   
Diplome                 1 - Below BD       169.8 (16.9%)  184.6 (18.5%)   
                        2 - BD             240.1 (23.9%)  196.7 (19.7%)   
                        3 - Above BD       593.1 (59.1%)  618.7 (61.9%)   
Zones                   Paris              158.3 (15.8%)  132.6 (13.3%)   
                        Rural              224.7 (22.4%)  224.0 (22.4%)   
                        Urban <100k        169.0 (16.8%)  151.1 (15.1%)   
                        Urban <20k         100.2 (10.0%)  115.4 (11.5%)   
                        Urban >100k        350.9 (35.0%)  377.0 (37.7%)   
Littéracie              1 - Adequate       604.8 (60.3%)  552.7 (55.3%)   
                        2 - Problematic    263.6 (26.3%)  296.0 (29.6%)   
                        3 - Inadequate     134.6 (13.4%)  151.2 (15.1%)   
Défavorisé              No                 643.0 (64.1%)  639.2 (63.9%)   
                        Yes                360.0 (35.9%)  360.8 (36.1%)   
Confiance scientifiques 1 - Yes            896.6 (89.4%)  870.2 (87.0%)   
                        2 - No             106.4 (10.6%)  129.8 (13.0%)   
Confiance médecins      1 - Yes            933.8 (93.1%)  920.9 (92.1%)   
                        2 - No               69.2 (6.9%)    79.1 (7.9%)   
Confiance politiques    1 - Yes            178.4 (17.8%)  152.1 (15.2%)   
                        2 - No             824.6 (82.2%)  847.9 (84.8%)   
Confiance industriels   1 - Yes            258.1 (25.7%)  422.0 (42.2%)   
                        2 - No             744.9 (74.3%)  578.0 (57.8%)   
Recherche info          No                 136.1 (13.6%)  159.8 (16.0%)   
                        Yes                866.9 (86.4%)  840.2 (84.0%)   
État Santé              1 - Good           649.2 (64.7%)  637.0 (63.7%)   
                        2 - Average        281.3 (28.0%)  289.9 (29.0%)   
                        3 - Bad              72.5 (7.2%)    73.1 (7.3%)   
Consultation médecin    False              345.9 (34.5%)  380.5 (38.0%)   
                        True               657.1 (65.5%)  619.5 (62.0%)   
Formation santé         No                 827.4 (82.5%)  778.0 (77.8%)   
                        Yes                175.6 (17.5%)  222.0 (22.2%)   
Perception Corona       No                   54.5 (5.4%)    61.4 (6.1%)   
                        Some               683.6 (68.2%)  622.9 (62.3%)   
                        Yes                264.9 (26.4%)  315.8 (31.6%)   
Revenus                 1 - Moins de 1000            NaN    52.7 (5.3%)   
                        2 - Moins de 2000            NaN  234.4 (23.4%)   
                        3 - Moins de 5000            NaN  532.0 (53.2%)   
                        4 - Plus de 5000             NaN    67.6 (6.8%)   
                        NSPR                         NaN  113.3 (11.3%)   

                                              All  
Age                     [18-35[             612.9  
                        [35-45[             378.6  
                        [45-55[             390.6  
                        [55-65[             364.5  
                        [65-75]             256.4  
Sexe                    female             1029.5  
                        male                973.5  
Diplome                 1 - Below BD        354.4  
                        2 - BD              436.9  
                        3 - Above BD       1211.8  
Zones                   Paris   

Finalement, ça a peu bougé. Deux options : 
- Fusionner les deux vagues ensemble pour gagner en puissance statistique
- Uniquement la vague 2 qui a le revenu ...

Pour le moment, uniquement vague 2

## Tri à plat et analyse

RM6 Les nouveaux médicaments sont toujours plus efficaces que les anciens

In [71]:
pyshs.tableau_croise_multiple(data[data["Vague"]==2],"efficacitenouveaumed",ind_var,weight="weight")

Accord  \
Variable                            Modalités                          
Age (p = 0.000)                     [18-35[            107.0 (35.0%)   
                                    [35-45[             40.8 (21.6%)   
                                    [45-55[             42.8 (21.9%)   
                                    [55-65[             34.4 (18.9%)   
                                    [65-75]             28.5 (22.3%)   
Sexe (p = 0.017)                    female             113.4 (22.1%)   
                                    male               140.0 (28.8%)   
Diplome (p = 0.404)                 1 - Below BD        48.1 (26.1%)   
                                    2 - BD              56.6 (28.8%)   
                                    3 - Above BD       148.8 (24.1%)   
Zones (p = 0.127)                   Paris               38.6 (29.1%)   
                                    Rural               51.4 (22.9%)   
                                    Urban <100k         30.5 (20.2%)   
                                    Urban <20k          24.5 (21.2%)   
                                    Urban >100k        108.5 (28.8%)   
Littéracie (p = 0.000)              1 - Adequate       169.3 (30.6%)   
                                    2 - Problematic     56.4 (19.0%)   
                                    3 - Inadequate      27.8 (18.4%)   
Défavorisé (p = 0.034)              No                 176.5 (27.6%)   
                                    Yes                 76.9 (21.3%)   
Confiance scientifiques (p = 0.915) 1 - Yes            219.6 (25.2%)   
                                    2 - No              33.9 (26.1%)   
Confiance médecins (p = 0.946)      1 - Yes            232.7 (25.3%)   
                                    2 - No              20.8 (26.3%)   
Confiance politiques (p = 0.000)    1 - Yes             68.7 (45.2%)   
                                    2 - No             184.7 (21.8%)   
Confiance industriels (p = 0.000)   1 - Yes            145.2 (34.4%)   
                                    2 - No             108.3 (18.7%)   
Recherche info (p = 0.003)          No                  55.8 (34.9%)   
                                    Yes                197.7 (23.5%)   
État Santé (p = 0.226)              1 - Good           168.6 (26.5%)   
                                    2 - Average         72.2 (24.9%)   
                                    3 - Bad             12.6 (17.3%)   
Consultation médecin (p = 0.371)    False               90.0 (23.6%)   
                                    True               163.5 (26.4%)   
Formation santé (p = 0.763)         No                 195.0 (25.1%)   
                                    Yes                 58.5 (26.4%)   
Perception Corona (p = 0.032)       No                  10.0 (16.3%)   
                                    Some               148.9 (23.9%)   
                                    Yes                 94.6 (30.0%)   
Revenus (p = 0.013)                 1 - Moins de 1000   17.8 (33.8%)   
                                    2 - Moins de 2000   57.3 (24.4%)   
                                    3 - Moins de 5000  135.5 (25.5%)   
                                    4 - Plus de 5000    25.1 (37.2%)   
                                    NSPR                17.7 (15.6%)   

                                                        Pas d'accord  \
Variable                            Modalités                          
Age (p = 0.000)                     [18-35[            199.0 (65.0%)   
                                    [35-45[            148.2 (78.4%)   
                                    [45-55[            152.2 (78.1%)   
                                    [55-65[            147.6 (81.1%)   
                                    [65-75]             99.5 (77.7%)   
Sexe (p = 0.017)                    female             400.6 (77.9%)   
                                    male               346.0 (71.2%)   
Diplome (p = 0.404)          

RM8 Il faut financer en priorité la recherche sur les nouvelles maladies par rapport à celles plus anciennes

In [64]:
pyshs.tableau_croise_multiple(data[data["Vague"]==2],"prioriteNouvellesMaladies",ind_var,weight="weight")

Agree  \
Variable                            Modalités                          
Age (p = 0.000)                     [18-35[            110.3 (36.0%)   
                                    [35-45[             54.6 (28.9%)   
                                    [45-55[             61.4 (31.5%)   
                                    [55-65[             77.9 (42.8%)   
                                    [65-75]             64.3 (50.2%)   
Sexe (p = 0.006)                    female             168.1 (32.7%)   
                                    male               200.5 (41.3%)   
Diplome (p = 0.000)                 1 - Below BD        94.7 (51.3%)   
                                    2 - BD              84.1 (42.8%)   
                                    3 - Above BD       189.8 (30.7%)   
Zones (p = 0.149)                   Paris               41.7 (31.5%)   
                                    Rural               71.4 (31.9%)   
                                    Urban <100k         56.5 (37.4%)   
                                    Urban <20k          46.0 (39.9%)   
                                    Urban >100k        153.0 (40.6%)   
Littéracie (p = 0.000)              1 - Adequate       239.5 (43.3%)   
                                    2 - Problematic     83.1 (28.1%)   
                                    3 - Inadequate      46.1 (30.5%)   
Défavorisé (p = 0.195)              No                 245.6 (38.4%)   
                                    Yes                123.0 (34.1%)   
Confiance scientifiques (p = 0.945) 1 - Yes            320.9 (36.9%)   
                                    2 - No              47.7 (36.7%)   
Confiance médecins (p = 0.569)      1 - Yes            336.6 (36.6%)   
                                    2 - No              32.0 (40.5%)   
Confiance politiques (p = 0.007)    1 - Yes             71.4 (46.9%)   
                                    2 - No             297.2 (35.1%)   
Confiance industriels (p = 0.000)   1 - Yes            186.2 (44.1%)   
                                    2 - No             182.4 (31.6%)   
Recherche info (p = 0.000)          No                  82.3 (51.5%)   
                                    Yes                286.3 (34.1%)   
État Santé (p = 0.979)              1 - Good           235.2 (36.9%)   
                                    2 - Average        107.3 (37.0%)   
                                    3 - Bad             26.1 (35.8%)   
Consultation médecin (p = 0.995)    False              139.8 (36.7%)   
                                    True               228.8 (36.9%)   
Formation santé (p = 0.785)         No                 284.5 (36.6%)   
                                    Yes                 84.1 (37.9%)   
Perception Corona (p = 0.129)       No                  22.1 (36.0%)   
                                    Some               215.9 (34.7%)   
                                    Yes                130.7 (41.4%)   
Revenus (p = 0.508)                 1 - Moins de 1000   24.7 (46.9%)   
                                    2 - Moins de 2000   89.8 (38.3%)   
                                    3 - Moins de 5000  187.2 (35.2%)   
                                    4 - Plus de 5000    26.3 (38.9%)   
                                    NSPR                40.5 (35.7%)   

                                                            Disagree  \
Variable                            Modalités                          
Age (p = 0.000)                     [18-35[            195.7 (64.0%)   
                                    [35-45[            134.4 (71.1%)   
                                    [45-55[            133.6 (68.5%)   
                                    [55-65[            104.1 (57.2%)   
                                    [65-75]             63.7 (49.8%)   
Sexe (p = 0.006)                    female             345.9 (67.3%)   
                                    male               285.5 (58.7%)   
Diplome (p = 0.000)           

RM3 Il faut laisser aux industriels la responsabilité de développer les nouveaux médicaments

In [56]:
pyshs.tableau_croise_multiple(data[data["Vague"]==2],"respindustrielsD",ind_var,weight="weight")

1 - Agree  \
Variable                            Modalités                          
Age (p = 0.209)                     [18-35[            136.1 (44.5%)   
                                    [35-45[             86.8 (45.9%)   
                                    [45-55[             80.8 (41.4%)   
                                    [55-65[             66.5 (36.5%)   
                                    [65-75]             62.3 (48.7%)   
Sexe (p = 0.045)                    female             206.1 (40.1%)   
                                    male               226.4 (46.6%)   
Diplome (p = 0.051)                 1 - Below BD        93.9 (50.9%)   
                                    2 - BD              86.1 (43.8%)   
                                    3 - Above BD       252.4 (40.8%)   
Zones (p = 0.336)                   Paris               66.8 (50.4%)   
                                    Rural               87.4 (39.0%)   
                                    Urban <100k         65.2 (43.2%)   
                                    Urban <20k          48.1 (41.7%)   
                                    Urban >100k        164.9 (43.7%)   
Littéracie (p = 0.000)              1 - Adequate       272.4 (49.3%)   
                                    2 - Problematic    107.3 (36.2%)   
                                    3 - Inadequate      52.8 (34.9%)   
Défavorisé (p = 0.006)              No                 297.7 (46.6%)   
                                    Yes                134.8 (37.4%)   
Confiance scientifiques (p = 0.208) 1 - Yes            383.4 (44.1%)   
                                    2 - No              49.0 (37.8%)   
Confiance médecins (p = 0.072)      1 - Yes            406.4 (44.1%)   
                                    2 - No              26.1 (33.0%)   
Confiance politiques (p = 0.000)    1 - Yes            100.7 (66.2%)   
                                    2 - No             331.7 (39.1%)   
Confiance industriels (p = 0.000)   1 - Yes            253.1 (60.0%)   
                                    2 - No             179.3 (31.0%)   
Recherche info (p = 0.005)          No                  85.6 (53.6%)   
                                    Yes                346.9 (41.3%)   
État Santé (p = 0.000)              1 - Good           305.7 (48.0%)   
                                    2 - Average        102.0 (35.2%)   
                                    3 - Bad             24.8 (33.9%)   
Consultation médecin (p = 0.126)    False              176.7 (46.4%)   
                                    True               255.8 (41.3%)   
Formation santé (p = 0.618)         No                 340.2 (43.7%)   
                                    Yes                 92.3 (41.6%)   
Perception Corona (p = 0.173)       No                  28.0 (45.6%)   
                                    Some               255.3 (41.0%)   
                                    Yes                149.2 (47.3%)   
Revenus (p = 0.038)                 1 - Moins de 1000   26.0 (49.3%)   
                                    2 - Moins de 2000   95.7 (40.8%)   
                                    3 - Moins de 5000  227.5 (42.8%)   
                                    4 - Plus de 5000    40.3 (59.7%)   
                                    NSPR                43.0 (38.0%)   

                                                        2 - Disagree  \
Variable                            Modalités                          
Age (p = 0.209)                     [18-35[            169.9 (55.5%)   
                                    [35-45[            102.2 (54.1%)   
                                    [45-55[            114.2 (58.6%)   
                                    [55-65[            115.5 (63.5%)   
                                    [65-75]             65.7 (51.3%)   
Sexe (p = 0.045)                    female             307.9 (59.9%)   
                                    male               259.6 (53.4%)   
Diplome (p = 0.051)       

RM4 L'Etat doit intervenir directement pour développer un nouveau médicament quand la maladie est grave

In [57]:
pyshs.tableau_croise_multiple(data[data["Vague"]==2],"respetatD",ind_var,weight="weight")

1 - Agree  \
Variable                            Modalités                          
Age (p = 0.297)                     [18-35[            258.2 (84.4%)   
                                    [35-45[            167.6 (88.7%)   
                                    [45-55[            167.1 (85.7%)   
                                    [55-65[            153.0 (84.1%)   
                                    [65-75]            116.3 (90.9%)   
Sexe (p = 0.298)                    female             437.0 (85.0%)   
                                    male               425.2 (87.5%)   
Diplome (p = 0.208)                 1 - Below BD       162.5 (88.0%)   
                                    2 - BD             162.2 (82.4%)   
                                    3 - Above BD       537.6 (86.9%)   
Zones (p = 0.570)                   Paris              108.6 (82.0%)   
                                    Rural              195.1 (87.1%)   
                                    Urban <100k        131.7 (87.2%)   
                                    Urban <20k          97.6 (84.6%)   
                                    Urban >100k        329.2 (87.3%)   
Littéracie (p = 0.003)              1 - Adequate       494.4 (89.5%)   
                                    2 - Problematic    246.9 (83.4%)   
                                    3 - Inadequate     121.0 (80.0%)   
Défavorisé (p = 0.010)              No                 565.1 (88.4%)   
                                    Yes                297.1 (82.3%)   
Confiance scientifiques (p = 0.017) 1 - Yes            759.6 (87.3%)   
                                    2 - No             102.7 (79.1%)   
Confiance médecins (p = 0.003)      1 - Yes            803.2 (87.2%)   
                                    2 - No              59.1 (74.7%)   
Confiance politiques (p = 0.015)    1 - Yes            141.3 (92.8%)   
                                    2 - No             721.0 (85.0%)   
Confiance industriels (p = 0.000)   1 - Yes            398.9 (94.5%)   
                                    2 - No             463.4 (80.2%)   
Recherche info (p = 0.884)          No                 136.7 (85.5%)   
                                    Yes                725.6 (86.4%)   
État Santé (p = 0.162)              1 - Good           557.1 (87.5%)   
                                    2 - Average        240.6 (83.0%)   
                                    3 - Bad             64.6 (88.4%)   
Consultation médecin (p = 0.650)    False              325.2 (85.5%)   
                                    True               537.0 (86.7%)   
Formation santé (p = 0.536)         No                 667.5 (85.8%)   
                                    Yes                194.8 (87.7%)   
Perception Corona (p = 0.053)       No                  49.1 (80.0%)   
                                    Some               530.0 (85.1%)   
                                    Yes                283.2 (89.7%)   
Revenus (p = 0.083)                 1 - Moins de 1000   44.6 (84.6%)   
                                    2 - Moins de 2000  191.5 (81.7%)   
                                    3 - Moins de 5000  465.5 (87.5%)   
                                    4 - Plus de 5000    63.5 (93.9%)   
                                    NSPR                97.1 (85.7%)   

                                                        2 - Disagree  \
Variable                            Modalités                          
Age (p = 0.297)                     [18-35[             47.8 (15.6%)   
                                    [35-45[             21.4 (11.3%)   
                                    [45-55[             27.9 (14.3%)   
                                    [55-65[             29.0 (15.9%)   
                                    [65-75]              11.7 (9.1%)   
Sexe (p = 0.298)                    female              77.0 (15.0%)   
                                    male                60.8 (12.5%)   
Diplome (p = 0.208)       

#### Croisement entre la responsabilité de l'état et des industriels

In [61]:
pyshs.tableau_croise(data[data["Vague"]==2], "respindustrielsD","respetatD","weight")

respetatD,1 - Agree,2 - Disagree,All
respindustrielsD,,,
1 - Agree,409.6 (94.7%),22.9 (5.3%),432.5
2 - Disagree,452.7 (79.8%),114.8 (20.2%),567.5
All,862.2,137.8,1000.0


Pas d'incompatibilité entre l'importance donné aux industriels et le rôle interventionniste de l'Etat. Le non interventionnisme en France est très peu présent

#### Croisement entre efficacité nouveaux médicaments et responsabilité des industriels

In [63]:
pyshs.tableau_croise(data[data["Vague"]==2], "efficacitenouveaumed","respindustrielsD","weight")

respindustrielsD,1 - Agree,2 - Disagree,All
efficacitenouveaumed,,,
Accord,180.4 (71.2%),73.0 (28.8%),253.5
Pas d'accord,252.0 (33.8%),494.5 (66.2%),746.5
All,432.5,567.5,1000.0


Peu de répondant qui trouvent que les médicaments nouveaux sont plus efficace et qui sont en désaccord avec le rôle des industriels

## Modèle

Sélection uniquement des variables les plus pertinentes

In [38]:
from scipy.stats.distributions import chi2

def construction_formule(dep,indep):
    """
    Construit une formule de modèle à partir d'une liste de variables
    """
    return dep + " ~ " + " + ".join(indep)

def likelihood_ratio(mod, mod_r):
    """
    Différence de déviance entre deux modèles [à revérifier]
    """
    val = [mod.llf,mod_r.llf]
    LR = 2*(max(val)-min(val)) #rapport de déviance
    
    val = [mod.df_model,mod_r.df_model]
    diff_df = max(val)-min(val) #différence de ddf

    p = chi2.sf(LR,diff_df) #test de la significativité
    return p

def pseudoR2(mod):
    R = 1-mod.deviance/mod.null_deviance
    return round(R,2)

def model(var_dep,ind_var,data,all_models=True):
    """
    Construction du modèle suivant un purposeful selection process (Agresti, 2013)
    """

    # Test des associations au chi2
    #t = corr_vars(data,var_dep,[i for i in ind_var],"RED") # Associations
    # Création de deux listes
    #v_asso = list(t[t["p value"]<0.20].index) # Variables associées
    #v_non_asso = list(t[t["p value"]>=0.20].index) # Variables non associées associées

    # Association univariée Wald
    v_non_asso = []
    v_asso = []
    for i in ind_var:
        f = construction_formule(var_dep,[i])
        modele = smf.glm(formula=f, data=data, family=sm.families.Binomial(), freq_weights=data["weight"])
        reg = modele.fit()
        p = reg.wald_test_terms().table.loc[i,"pvalue"]
        if p<0.2:
            v_asso.append(i)
        else:
            v_non_asso.append(i)
    print(v_asso)
    
    # Construction du premier modèle complet comme point de départ
    ind_var_mod = v_asso.copy()
    f = construction_formule(var_dep,ind_var_mod)
    modele = smf.glm(formula=f, data=data, family=sm.families.Binomial(), freq_weights=data["weight"])
    regression_ref = modele.fit()
    m1 = regression_ref  #modèle de référence initial
    print(pseudoR2(regression_ref))

    # Liste des variables non significatives test de Wald
    t = regression_ref.wald_test_terms().table
    var_test = list(t[t["pvalue"]>=0.1].index)
    if "Intercept" in var_test:
        var_test.remove("Intercept")

    # Pour chaque variable non significative, l'enlever et voir si cela change le modèle
    for v in var_test:
        
        # Construire le modèle sans la variable
        f_t = construction_formule(var_dep,[i for i in ind_var_mod if i != v])
        modele = smf.glm(formula=f_t, 
            data=data, family=sm.families.Binomial(), freq_weights=data["weight"])
        regression = modele.fit()
        
        # Comparer le ratio de vraissemblance par rapport au modèle avec la variable
        p = likelihood_ratio(regression_ref,regression)

        # Si cela ne modifie pas significativement le modèle, enlever la variable et redéfinir le modèle de ref
        if p > 0.05:
            print("remove %s"%v)
            ind_var_mod.remove(v)
            regression_ref = regression

    # Modèle intermédiaire 
    m2 = regression_ref

    # Ajouter une à une les variables initialement supprimées pour voir si elles améliorent
    for v in v_non_asso:
        
        # Construire le modèle
        f_t = construction_formule(var_dep,ind_var_mod+[v])
        modele = smf.glm(formula=f_t, 
            data=data, family=sm.families.Binomial(), freq_weights=data["weight"])
        regression = modele.fit()
        p = likelihood_ratio(regression_ref,regression)
        
        # Si cela ajoute, modifier le modèle
        if p <= 0.05:
            print("add %s"%v)
            ind_var_mod.append(v)
            regression_ref = regression

    m3 = regression_ref
    print(pseudoR2(regression_ref))
    f_t = construction_formule(var_dep,ind_var_mod)
    print(f_t)
    if all_models:
        return pd.concat([pyshs.presentation_logistique(m1,sig=True),
                          pyshs.presentation_logistique(m2, sig=True),
                          pyshs.presentation_logistique(m3, sig=True)],axis=1)
    else:
        return m3
    

Modèle sur le fait de trouver un médicament nouveau plus efficace

In [40]:
m_eff = model("efficacitenouveaumedB",ind_var,data=data[data["Vague"]==2],all_models=False)
pyshs.presentation_logistique(m_eff,sig=True)

['age2', 'sexe', 'zones', 'litQ2', 'DEF', 'confPolitiques', 'confIndustriels', 'rechercheinfo', 'PERCEPTCORONA', 'rev']
0.1
remove zones
remove DEF
remove rev
add confScientifiques
0.1
efficacitenouveaumedB ~ age2 + sexe + litQ2 + confPolitiques + confIndustriels + rechercheinfo + PERCEPTCORONA + confScientifiques


,odd ratio,p-value,IC
Intercept,0.47,0.107,0.47 [0.19-1.18]
age2[T.[35-45[],0.47,0.001***,0.47 [0.30-0.74]
age2[T.[45-55[],0.53,0.005***,0.53 [0.34-0.82]
age2[T.[55-65[],0.43,0.0***,0.43 [0.27-0.68]
age2[T.[65-75]],0.49,0.005***,0.49 [0.29-0.80]
sexe[T.male],1.47,0.015**,1.47 [1.08-2.00]
litQ2[T.T2],1.25,0.243,1.25 [0.86-1.82]
litQ2[T.T3],2.49,0.0***,2.49 [1.69-3.68]
confPolitiques[T.2 - No],0.56,0.006***,0.56 [0.37-0.85]
confIndustriels[T.2 - No],0.59,0.002***,0.59 [0.42-0.82]
